# ⚠️ PyKwant Tutorial: Risk & Sensitivities

This notebook introduces the `pykwant.risk` module.

In many libraries, every instrument class must implement its own `duration()` method. In PyKwant, we take a generic numerical approach:

* We treat the Pricing Logic as a function $P(r)$.

* We calculate sensitivities (Greeks) by numerically differentiating this function.

This means our Risk module works automatically for **any** instrument that can be priced, without writing specific analytical formulas for each one.

## 1. Setup and Imports

In [ ]:
import math
from datetime import date

from pykwant import dates, instruments, risk


# Helper to format output
def print_metric(name, value, unit=""):
    print(f"{name:<20}: {value:>12.6f} {unit}")

## 2. Market & Instrument Setup

We will analyze the risk of a **10-Year Bond** in a **5% Rate Environment**.

In [2]:
# 1. Calendar
cal = dates.Calendar(holidays=frozenset(), weekends=(6, 7))

# 2. Instrument: 10Y Bond, 4% Coupon
bond_10y = instruments.FixedRateBond(
    face_value=instruments.Money(100.0),
    coupon_rate=0.04,
    start_date=date(2025, 1, 1),
    maturity_date=date(2035, 1, 1),
    frequency_months=12,
    day_count=dates.thirty_360,
    calendar=cal,
)

# 3. Market: Flat 5% Curve
val_date = date(2025, 1, 1)


def flat_curve(d: date) -> float:
    t = dates.act_365(val_date, d)
    return math.exp(-0.05 * t)


# Base Price
base_price = instruments.price_instrument(bond_10y, flat_curve, val_date)
print(f"Base Price: {base_price:.4f}")

Base Price: 91.3298


## 3. PV01 (Sensitivity to 1bp)

The PV01 measures the absolute change in price for a **1 basis point (+0.01%)** parallel shift in the yield curve.

$$PV01 \approx P(r + 1bp) - P(r)$$

The `risk` module handles the curve shifting internally using closures.

In [3]:
pv01_val = risk.pv01(instrument=bond_10y, curve=flat_curve, valuation_date=val_date)

print_metric("PV01", pv01_val)

# Interpretation:
# If rates go UP by 1bp, the bond price goes DOWN by ~0.07.

PV01                :    -0.076261 


## 4. Effective Duration

**Effective Duration** is the percentage sensitivity of the price to interest rates. It approximates the first derivative of the price function.

$$D_{eff} = - \frac{1}{P} \frac{dP}{dr}$$

Because this is calculated numerically, it works correctly even for instruments with complex cash flows (like amortizing bonds).

In [4]:
duration = risk.effective_duration(bond_10y, flat_curve, val_date)

print_metric("Effective Duration", duration, "Years")

# Check Approximation:
# Price Change % ≈ -Duration * Rate Change
# Price Change   ≈ -Duration * Price * Rate Change
estimated_change = -duration * base_price * 0.0001
print_metric("Est. Change (1bp)", estimated_change)
print_metric("Actual PV01", pv01_val)

Effective Duration  :     8.354012 Years
Est. Change (1bp)   :    -0.076297 
Actual PV01         :    -0.076261 


## 5. Effective Convexity

**Convexity** measures the curvature of the price-yield relationship. It improves the accuracy of price estimates for large rate moves.

$$C_{eff} = \frac{1}{P} \frac{d^2P}{dr^2}$$

In [5]:
convexity = risk.effective_convexity(bond_10y, flat_curve, val_date)

print_metric("Effective Convexity", convexity)

Effective Convexity :    77.918852 


## 6. Comprehensive Risk Report

The `calculate_risk_metrics` function aggregates all these measures into a single dictionary, which is efficient for generating reports.

In [6]:
print("\n--- Full Risk Report ---")
report = risk.calculate_risk_metrics(bond_10y, flat_curve, val_date)

for k, v in report.items():
    print_metric(k.replace("_", " ").title(), v)


--- Full Risk Report ---
Price               :    91.329824 
Duration            :     8.354012 
Convexity           :    77.918852 
Dv01                :    -0.076261 


## 7. Analyzing Large Moves (Convexity Bias)

Why do we care about Convexity? For small moves (1bp), Duration is enough. For large moves (e.g., +200bps or +2%), the linear approximation fails.

Let's simulate a **+2.00%** shock.

In [ ]:
shock = 0.02

# 1. Linear Estimate (Duration only)
# dP/P ≈ -D * dr
linear_est = base_price * (-duration * shock)

# 2. Quadratic Estimate (Duration + Convexity)
# dP/P ≈ -D * dr + 0.5 * C * (dr)^2
quad_est = base_price * (-duration * shock + 0.5 * convexity * (shock**2))


# 3. Actual Repricing
# We manually shift the curve to check the "truth"
def shocked_curve(d):
    # Original 5% + 2% shock = 7%
    t = dates.act_365(val_date, d)
    return math.exp(-0.07 * t)


new_price = instruments.price_instrument(bond_10y, shocked_curve, val_date)
actual_diff = new_price - base_price

print("\n--- Scenario: Rates +2.00% ---")
print_metric("Linear Estimate", linear_est)
print_metric("Quadratic Est.", quad_est)
print_metric("Actual Change", actual_diff)

error_linear = abs(actual_diff - linear_est)
error_quad = abs(actual_diff - quad_est)

print(f"\nError (Linear):    {error_linear:.4f} (Too pessimistic)")
print(f"Error (Quadratic): {error_quad:.4f} (Much closer)")


--- Scenario: Rates +2.00% ---
Linear Estimate     :   -15.259410 
Quadratic Est.      :   -13.836147 
Actual Change       :   -13.923476 

Error (Linear):    1.3359 (Too pessimistic)
Error (Quadratic): 0.0873 (Much closer)
